# Surfa Data Preparation Notebook

This notebook 
* reads in the the raw data, 
* applies appropriate transformations, 
* executes the appropriate feature engeneering, 
* cleans the data and
* stores it in a dataset that can be used in training the model.

In [216]:
import pandas as pd
import numpy as np

In [217]:
tips = pd.read_csv('raw_data/raw_tips.csv', parse_dates=False)
tips.info()
tips.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 5 columns):
Date        101 non-null object
Tip         101 non-null float64
Orders      101 non-null int64
Distance    101 non-null float64
DriverID    101 non-null int64
dtypes: float64(2), int64(2), object(1)
memory usage: 4.0+ KB


,Date,Tip,Orders,Distance,DriverID
0,2021-01-10,0.00,8,28.0,1
1,2021-01-02,0.00,5,13.9,1
2,2021-01-13,0.00,8,20.0,1
3,2021-01-16,0.00,10,30.0,1
4,2021-01-21,2.45,11,37.5,1


## Auditing and cleaning

In [218]:
def audit_surfa(df):
    print('Start audit')
    assure_num_type(df,'Tip', float, min=0, max=200)
    assure_num_type(df,'Distance', float, min=0, max=100)
    assure_num_type(df,'Orders', int, min=0, max=50)
    assure_num_type(df,'DriverID', int, min=0)
    assure_date(df, 'Date', min='2020-09-01', max='2021-03-18')
    assure_mandatory(df, df.columns)
    assure_orders_exist_for_tips(df)
    print('Audit completed')

def assure_num_type(df, col, dtype, min= None, max=None):
    df[col] = df[col].astype(dtype)

    assure_min(df, col, min)
    assure_max(df, col, max)

def assure_min(df, col, min):
    lower = df[col] < min
    if lower.any(): 
        print('Fail! #ValueTooLow', col, '\n', df.loc[lower])

def assure_max(df, col, max):
    higher = df[col] > max
    if higher.any(): 
        print('Fail! #ValueTooHigh', col, '\n', df.loc[higher])

def assure_date(df, col, min=None, max=None):
    try:
        df[col] = pd.to_datetime(df[col], format='%Y-%m-%d', errors='raise')
    except ValueError as error:
        print('Fail! #DateFormatWrong (expected yyyy-mm-dd)', col, '\n', error)

    assure_min(df, col, min)
    assure_max(df, col, max)

def assure_mandatory(df, cols):

    invalid = df[cols].isna().any(axis=1)
    if invalid.any():
        print('Fail! #MissingValues ', cols, '\n', df.loc[invalid] )

def assure_orders_exist_for_tips(df):
    invalid = (df['Tip'] > 0) & (df['Orders'] == 0)
    if invalid.any():
        print('Warning! #NoOrdersForTip \n', df.loc[invalid])
    return invalid


audit_surfa(tips)

Start audit
Warning! #NoOrdersForTip 
          Date   Tip  Orders  Distance  DriverID
37 2021-01-31  2.32       0       0.0         2
Audit completed


In [219]:
def autofix_data(df: pd.DataFrame) -> pd.DataFrame:
    todrop = assure_orders_exist_for_tips(df)
    df = df.loc[~todrop]

    print('Autofixing finished')
    return df

tips = autofix_data(tips)


Warning! #NoOrdersForTip 
          Date   Tip  Orders  Distance  DriverID
37 2021-01-31  2.32       0       0.0         2
Autofixing finished


# Enreaching tips dataset

Add `TipPerOrder` as label to deal properly with drivers having double shifts.

In [220]:
tips['TipPerOrder'] = tips['Tip'] / tips['Orders']
tips.head(5)

,Date,Tip,Orders,Distance,DriverID,TipPerOrder
0,2021-01-10,0.00,8,28.0,1,0.000000
1,2021-01-02,0.00,5,13.9,1,0.000000
2,2021-01-13,0.00,8,20.0,1,0.000000
3,2021-01-16,0.00,10,30.0,1,0.000000
4,2021-01-21,2.45,11,37.5,1,0.222727


Add day of week and whether hte shift is during a weekend or not.

In [221]:
tips['DayOfWeek'] = tips['Date'].dt.dayofweek
tips['IsWeekend'] = (tips['DayOfWeek'] > 3).astype(float) # 4, 5, 6 are Fri, Sat, Sun respectievely, and considered weekend
tips.head(5)

,Date,Tip,Orders,Distance,DriverID,TipPerOrder,DayOfWeek,IsWeekend
0,2021-01-10,0.00,8,28.0,1,0.000000,6,1.0
1,2021-01-02,0.00,5,13.9,1,0.000000,5,1.0
2,2021-01-13,0.00,8,20.0,1,0.000000,2,0.0
3,2021-01-16,0.00,10,30.0,1,0.000000,5,1.0
4,2021-01-21,2.45,11,37.5,1,0.222727,3,0.0


# KNMI Weather Data 

In [222]:
weather = pd.read_csv('./raw_data/raw_weather.csv', parse_dates=['YYYYMMDD'], index_col=['YYYYMMDD'], na_values='     ', header=44, dtype=float, converters = {'YYYYMMDD': str} )
print(weather.shape)
weather = weather.loc['2020-01-01':] \
            .drop(columns=['STN']) \
            .rename(columns=str.strip) \
            .rename(columns={
                'FG': 'MeanWind',
                'FHX': 'MaxWind',
                'FHXH': 'MaxWindHour',
                'FHN': 'MinWind',
                'FHNH': 'MinWindHour',
                'FXX': 'MaxWindGust',
                'FXXH': 'MaxWindGustHour',
                'TG': 'MeanTemp',
                'TN': 'MinTemp',
                'TNH': 'MinTempHour',
                'TX': 'MaxTemp',
                'TXH': 'MaxTempHour',
                'SQ': 'SunshineDur',
                'DR': 'RainDur',
                'RH': 'RainAmount',
                'RHX': 'MaxRainAmount',
                'RHXH': 'MaxRainAmountHour',
                'PG': 'MeanPress',
                'PX': 'MaxPress',
                'PXH': 'MaxPressHour',
                'PN': 'MinPress',
                'PNH': 'MinPressHour',
                'VVN': 'MinVis',
                'VVNH': 'MinVisHour',
                'VVX': 'MaxVis',
                'VVXH': 'MaxVisHour',
                'NG': 'Cloudness',
                'UG': 'MeanHum',
                'UX': 'MaxHum',
                'UXH': 'MaxHumHour',
                'UN': 'MinHum',
                'UNH': 'MinHumHour',
            })
weather.info()
weather.loc[:,].head(5)

<ipython-input-222-e57315947825>:1: ParserWarning: Both a converter and dtype were specified for column YYYYMMDD - only the converter will be used
  weather = pd.read_csv('./raw_data/raw_weather.csv', parse_dates=['YYYYMMDD'], index_col=['YYYYMMDD'], na_values='     ', header=44, dtype=float, converters = {'YYYYMMDD': str} )


(25644, 40)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 442 entries, 2020-01-01 to 2021-03-17
Data columns (total 39 columns):
DDVEC                442 non-null float64
FHVEC                442 non-null float64
MeanWind             442 non-null float64
MaxWind              442 non-null float64
MaxWindHour          442 non-null float64
MinWind              442 non-null float64
MinWindHour          442 non-null float64
MaxWindGust          442 non-null float64
MaxWindGustHour      442 non-null float64
MeanTemp             442 non-null float64
MinTemp              442 non-null float64
MinTempHour          442 non-null float64
MaxTemp              442 non-null float64
MaxTempHour          442 non-null float64
T10N                 442 non-null float64
T10NH                442 non-null float64
SunshineDur          442 non-null float64
SP                   442 non-null float64
Q                    442 non-null float64
RainDur              442 non-null float64
RainAmount           442

,DDVEC,FHVEC,MeanWind,MaxWind,MaxWindHour,MinWind,MinWindHour,MaxWindGust,MaxWindGustHour,MeanTemp,...,MinVisHour,MaxVis,MaxVisHour,Cloudness,MeanHum,MaxHum,MaxHumHour,MinHum,MinHumHour,EV24
YYYYMMDD,,,,,,,,,,,,,,,,,,,,,
2020-01-01,159.0,16.0,21.0,40.0,24.0,10.0,17.0,70.0,24.0,6.0,...,10.0,37.0,1.0,8.0,97.0,98.0,6.0,92.0,1.0,1.0
2020-01-02,182.0,37.0,38.0,50.0,19.0,20.0,4.0,90.0,19.0,37.0,...,1.0,70.0,16.0,8.0,94.0,98.0,1.0,87.0,23.0,2.0
2020-01-03,232.0,40.0,53.0,70.0,9.0,30.0,19.0,120.0,14.0,76.0,...,17.0,70.0,1.0,7.0,91.0,96.0,14.0,84.0,8.0,1.0
2020-01-04,249.0,43.0,45.0,60.0,9.0,20.0,22.0,100.0,12.0,56.0,...,19.0,74.0,14.0,8.0,92.0,97.0,21.0,80.0,14.0,3.0
2020-01-05,213.0,32.0,32.0,40.0,11.0,30.0,1.0,60.0,13.0,65.0,...,4.0,75.0,13.0,8.0,87.0,95.0,1.0,77.0,13.0,1.0


In [223]:
df  = tips.join(weather, on='Date')
df.head(5)

,Date,Tip,Orders,Distance,DriverID,TipPerOrder,DayOfWeek,IsWeekend,DDVEC,FHVEC,...,MinVisHour,MaxVis,MaxVisHour,Cloudness,MeanHum,MaxHum,MaxHumHour,MinHum,MinHumHour,EV24
0,2021-01-10,0.00,8,28.0,1,0.000000,6,1.0,208.0,20.0,...,1.0,65.0,14.0,6.0,94.0,99.0,1.0,79.0,14.0,4.0
1,2021-01-02,0.00,5,13.9,1,0.000000,5,1.0,190.0,16.0,...,20.0,59.0,1.0,8.0,95.0,98.0,17.0,92.0,2.0,1.0
2,2021-01-13,0.00,8,20.0,1,0.000000,2,0.0,250.0,36.0,...,23.0,73.0,14.0,8.0,89.0,96.0,24.0,78.0,14.0,4.0
3,2021-01-16,0.00,10,30.0,1,0.000000,5,1.0,182.0,32.0,...,17.0,81.0,13.0,8.0,86.0,98.0,17.0,63.0,14.0,2.0
4,2021-01-21,2.45,11,37.5,1,0.222727,3,0.0,203.0,74.0,...,24.0,83.0,1.0,7.0,78.0,95.0,19.0,58.0,2.0,4.0


In [224]:
df.to_csv('surfa_combined_dataset.csv', index=None)